### Import reviews

In [ ]:
%load_ext autoreload
%autoreload 2

from src.utils import log_progress as lp

In [ ]:
import pickle

DATA_FILE1 = 'data/data.pkl'
DATA_FILE2 = 'data/labels.pkl'
DATA_FILE3 = 'data/pos_adjectives_list.pkl'
DATA_FILE4 = 'data/neu_adjectives_list.pkl'
DATA_FILE5 = 'data/neg_adjectives_list.pkl'

with open(DATA_FILE1, 'rb') as f:
    reviews = pickle.load(f)
    
with open(DATA_FILE2, 'rb') as f:
    stars = pickle.load(f)

with open(DATA_FILE3, 'rb') as f:
    pos_adjectives_list = pickle.load(f)
    
with open(DATA_FILE4, 'rb') as f:
    neu_adjectives_list = pickle.load(f)

with open(DATA_FILE5, 'rb') as f:
    neg_adjectives_list = pickle.load(f)
    
entity_names = ['X', 'Y', 'Z', 'T', 'P', 'Q']

In [ ]:
print(pos_adjectives_list)

In [ ]:
from babble.babbler import BabbleStream

babbler = BabbleStream(reviews,stars,balanced=True, shuffled=True, seed=456)

In [ ]:
babbler.add_aliases({'positive': pos_adjectives_list})
babbler.add_aliases({'neutral': neu_adjectives_list})
babbler.add_aliases({'negative': neg_adjectives_list})

In [ ]:
babbler.semparser.grammar.lexical_rules

In [ ]:
from babble.utils import display_candidate

candidate = babbler.next()
print(candidate)
print(candidate.text)
print(candidate.entity_positions)

In [ ]:
print(dir(candidate.entities))
print(candidate.entities)
print(candidate.entities.__getitem__(0))

In [ ]:
'''
praticamente posso fare cosi: per ogni entità da X a Q, verifico se ci sono aggettivi appena prima o appena dopo(
da capire se può creare problemi") e creo in modo automatico la explanation.
Forse è il caso di provare prima con il solo check dell'aggettivo appena prima"
'''
from babble import Explanation
explanation = Explanation(
        name='LF_1',
        label=1,
        condition='the word "mediocre" is before X or the word "bad" is before Y or the word "good" is after T',
        candidate=candidate
    )

In [ ]:
import re

def get_words_before(sentence,entity):
    sentence = re.sub(r'[^\w\s]','',sentence)
    words = sentence.split()
    if entity in words:
        index = words.index(entity)
        before = index - min(index, 3)
        return words[before:index]  

In [ ]:
def check_adjectives_before(sentence,entity):
    words = get_words_before(sentence,entity)
    if words == None:
        return None
    else:
        for word in words:
            if word in pos_adjectives_list:
                return 'positive'
            elif word in neu_adjectives_list:
                return 'neutral'
            elif word in neg_adjectives_list:
                return 'negative'
        return None

In [ ]:
print(get_words_before(candidate.text,"salad"))

In [ ]:
print(check_adjectives_before(candidate.text,"salad"))

In [ ]:
from babble import Explanation

explanations = []

for candidate in reviews[0]:
    for entity,name in zip(candidate.entities,entity_names):
        adjectives_value = check_adjectives_before(candidate.text,entity.entity)
        if adjectives_value != None:
            explanation = Explanation(
            name='LF_1',
            label=1,
            condition='A ' + adjectives_value + ' word is before ' + name,
            candidate=candidate
            )
            explanations.append(explanation)

In [ ]:
print(explanations)

In [ ]:
babbler.view_parse(parses[0])

In [ ]:
print(parses[0].semantics)

In [ ]:
parses, filtered = babbler.apply(explanations)

In [ ]:
import spacy

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('en_core_web_sm')

In [ ]:
from babble import Explanation

final_parses = []

index = 0

for element,label,selected_words in (zip(lp.log_progress(reviews[0]),stars[0],tokens_list)):
    doc = spacy_nlp(element.text)
    tokens = [token.text for token in doc if not token.is_stop and not token.pos_  in ('PUNCT', 'NUM', 'SYM')]
        
    print(selected_words)
    
    explanation = Explanation(
        name='LF_' + str(index),
        label=int(label),
        condition=create_condition(selected_words),
        candidate=element
    )
    parses, filtered = babbler.apply(explanation)
    babbler.commit()
    
    index = index + 1

In [ ]:
babbler.commit()

In [ ]:
from metal.analysis import lf_summary

Ls = [babbler.get_label_matrix(split) for split in [0,1,2]]
lf_names = [lf.__name__ for lf in babbler.get_lfs()]
lf_summary(Ls[1], stars[1], lf_names=lf_names)

In [ ]:
print(Ls)

In [ ]:
from babble.utils import ExplanationIO

explanations = babbler.get_explanations()

FILE = "my_explanations.tsv"
exp_io = ExplanationIO()
exp_io.write(explanations, FILE)
explanations = exp_io.read(FILE)